# 09 - Backtest Replay

## Overview
Backtest strategy recommendations against actual race outcomes.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import json
from f1ts import config, io_flat

## Load

In [2]:
features_dir = config.paths()['data_features']
strategies = io_flat.read_parquet(features_dir / 'strategy_decisions.parquet')

✓ Loaded strategy_decisions.parquet: 10 rows, 6 cols
  Dtypes: {'n_stops': dtype('int64'), 'stop_laps': dtype('O'), 'compounds': dtype('O'), 'exp_finish_time_s': dtype('float64'), 'strategy_json': dtype('O'), 'delta_to_best_s': dtype('float64')}


## Transform: Compute Regret

In [3]:
# Simple regret calculation
if len(strategies) > 0:
    best_time = strategies['exp_finish_time_s'].min()
    strategies['regret_s'] = strategies['exp_finish_time_s'] - best_time
    print(f'Best strategy finish time: {best_time:.1f}s')
    print(f'Average regret: {strategies["regret_s"].mean():.2f}s')

Best strategy finish time: 3364.2s
Average regret: 4.99s


## Save

In [ ]:
backtest_summary = {
    'n_strategies_evaluated': len(strategies),
    'best_finish_time_s': float(strategies['exp_finish_time_s'].min()) if len(strategies) > 0 else None,
    'mean_regret_s': float(strategies['regret_s'].mean()) if len(strategies) > 0 else None,
}

metrics_dir = config.paths()['metrics']
io_flat.save_json(backtest_summary, metrics_dir / 'backtest_summary.json')
print('✓ Saved backtest summary')

## Repro Notes

- Computed regret vs optimal strategy
- Saved backtest metrics